In [ ]:
import pandas as pd

In [ ]:
path = '/Users/bablidey/Documents/KOAA/explore'

In [ ]:
import pandas as pd
import os
import csv

# Function to detect delimiter and inspect column count
def inspect_csv(file_path):
    with open(file_path, 'r') as csvfile:
        lines = csvfile.readlines()
        delimiter = csv.Sniffer().sniff(lines[7]).delimiter
        num_cols = len(lines[7].split(delimiter))
        return delimiter, num_cols, lines  # Return lines as well

# Directory containing the CSV files
directory_path = '/Users/babLidey/Documents/KOAA/explore'

# Initialize an empty DataFrame to hold all the data
all_data = pd.DataFrame()

# Loop through each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        
        # Detect the delimiter and number of columns, and get lines
        delimiter, num_cols, lines = inspect_csv(file_path)
        
        try:
            # Read the CSV file, skipping the metadata rows
            df = pd.read_csv(file_path, delimiter=delimiter, header=None, skiprows=7)
            
            # Handle files with different column counts
            if num_cols > 6:  # Assuming 6 is the correct number of useful data columns
                df = df.iloc[:, :6]  # Limit to the first 6 columns if more are found
            
            # Extract the region name from the lines
            region = lines[1].split('KOAA GA4 ')[1].strip()
            
            # Define the column names based on actual data columns present
            df.columns = ['Month', 'Year', 'Active Users', 'Total Users', 'Sessions', 'Views'][:df.shape[1]]
            
            # Insert the 'Region' column
            df.insert(0, 'Region', region)
            
            # Append to the all_data DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)
        
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

# Handle non-finite values before conversion
all_data.fillna(0, inplace=True)

# Convert columns to the correct datatype
try:
    all_data['Month'] = all_data['Month'].astype(int)
    all_data['Year'] = all_data['Year'].astype(int)
    all_data['Total Users'] = all_data['Total Users'].astype(int)
    all_data['Sessions'] = all_data['Sessions'].astype(int)
    all_data['Views'] = all_data['Views'].astype(int)
except Exception as e:
    print(f"Error converting data types: {e}")

# Save the combined data to a new CSV file
all_data.to_csv('combined_data.csv', index=False)

print("Data combined and saved to 'combined_data.csv'.")


In [ ]:
df = pd.read_csv('combined_data.csv')

In [ ]:
df

In [ ]:
## Removeing the first line from every region - i.e. Totals
df = df[df.duplicated(subset=['Region'],keep='first')]

In [ ]:
df.sort_values(by=['Region','Month'],inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.columns

In [ ]:
df.drop(columns='index',inplace=True)

In [ ]:
df

In [ ]:
# months = {
#     1: 'Jan-24',
#     2: 'Feb-24',
#     3: 'Mar-24',
#     4: 'Apr-24',
#     5: 'May-24',
#     6: 'Jun-24',
#     7: 'Jul-24'
# }

In [ ]:
# df['month'] = df['Month'].map(months)

In [ ]:
# df.drop(columns = ['Month','Year'],inplace=True)

In [ ]:
df

In [ ]:
df_set = set(df['Region'].unique())
col_set = set(['Rochester', 'Albany', 'Fairfield County', 'Buffalo',
'Houston', 'Salt Lake', 'Ann Arbor/Detroit', 'St. Louis', 'Westchester',
'Hudson Valley', 'Hartford', 'Denver', 'Austin', 'San Antonio',
'Nashville', 'MSP', 'Atlanta', 'Phoenix', 'Research Triangle',
'Charlotte', 'Dallas', 'DMV', 'Ft Worth', 'Midcities', 'Indianapolis',
'Jacksonville', 'Kansas City', 'Memphis', 'Milwaukee',
'Nassau / Long Island', 'Philadelphia', 'Pittsburgh', 'Providence',
'Suffolk', 'Tampa', 'Toronto', 'Vancouver', 'Inland Empire',
'Los Angeles', 'Orange County', 'San Diego', 'San Fernando',
'San Francisco', 'San Jose', 'Portland', 'Seattle'])

intersection = df_set and col_set
need_to_rename = df_set - intersection

In [ ]:
need_to_rename

In [ ]:
rename_region = {
    'Ann Arbor - Detroit': 'Ann Arbor/Detroit',
    'Fort Worth' : 'Ft Worth',
    'Long Island (Nassau)' : 'Nassau / Long Island',
    'Philly': 'Philadelphia',
    'Saint Louis':'St. Louis',
    'Salt Lake City' : 'Salt Lake'
}

df['Region']=df['Region'].replace(rename_region)

In [ ]:
df

In [ ]:
#df.set_index('Region',inplace=True)

In [ ]:
user_group=df.groupby(['Year','Month', 'Region'], as_index=False).sum()
users=user_group.pivot(index='Month',columns='Region',values='Total Users')
users=users.iloc[0:7,]

In [ ]:
session_group=df.groupby(['Year','Month', 'Sessions'], as_index=False).sum()
sessions=session_group.pivot(index='Month',columns='Region',values='Sessions')
sessions=sessions.iloc[0:7,]

In [ ]:
view_group=df.groupby(['Year','Month', 'Views'], as_index=False).sum()
views=view_group.pivot(index='Month',columns='Region',values='Views')
views=views.iloc[0:7,]

In [ ]:
users_per_month = pd.read_excel('final_report.xlsx',sheet_name='users_per_month')
sessions_report = pd.read_excel('final_report.xlsx',sheet_name='sessions_report')
view_report = pd.read_excel('final_report.xlsx',sheet_name='pageviews_report')
users_per_year = pd.read_excel('final_report.xlsx',sheet_name='users_per_year')

In [ ]:
users_per_month.columns

In [ ]:
users_per_year

In [ ]:
from datetime import datetime

users_per_month['Unnamed: 0'] = pd.to_datetime(users_per_month['Unnamed: 0']).dt.strftime('%b %Y')
sessions_report['Unnamed: 0'] = pd.to_datetime(sessions_report['Unnamed: 0']).dt.strftime('%b %Y')
view_report['Unnamed: 0']= pd.to_datetime(view_report['Unnamed: 0']).dt.strftime('%b %Y')
users_per_year['Month'] = pd.to_datetime(users_per_year['Month']).dt.strftime('%b %Y')

In [ ]:
users_per_month

In [ ]:
users_per_month.set_index('Unnamed: 0',inplace=True)
sessions_report.set_index('Unnamed: 0',inplace=True)
view_report.set_index('Unnamed: 0',inplace=True)


In [ ]:
users = users[users_per_month.columns]
sessions=sessions[sessions_report.columns]
views = views[view_report.columns]

In [ ]:
months = {
    1: 'Jan 2024',
    2: 'Feb 2024',
    3: 'Mar 2024',
    4: 'Apr 2024',
    5: 'May 2024',
    6: 'Jun 2024',
    7: 'Jul 2024'
}

In [ ]:
users.index = users.index.map(months)
sessions.index = sessions.index.map(months)
views.index = views.index.map(months)

In [ ]:
users_per_month = pd.concat([users_per_month,users])
sessions_report = pd.concat([sessions_report,users])
view_report = pd.concat([view_report,users])

In [ ]:
users

In [ ]:
sessions_report

In [ ]:
with pd.ExcelWriter('KOAA_Final_Report.xlsx') as w:
    users_per_month.to_excel(w,sheet_name='users_per_month', index = True)
    sessions_report.to_excel(w,sheet_name='session_report', index = True)
    view_report.to_excel(w, sheet_name='pageviews_report', index = True)
    users_per_year.to_excel(w, sheet_name='users_per_year', index = True)